In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yohanams","key":"77c3f279751c275ded9af956b0a12084"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
#upload file kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                                  title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
teocalvo/teomewhy-loyalty-system                                     TeoMeWhy Loyalty System                           22MB  2024-06-17 11:29:46            161        361  1.0              
shreyanshverma27/online-sales-dataset-popular-marketplace-data       Online Sales Dataset - Popular Marketplace Data    7KB  2024-05-25 23:55:26           7957        143  1.0              
damirdizdarevic/uefa-euro-2024-players                               UEFA EURO 2024 - Players                          15KB  2024-06-08 18:07:29           1564         37  1.0              
rabieelkharoua/students-performance-dataset       

In [ ]:
! kaggle datasets download -d paramaggarwal/fashion-product-images-small

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): copyright-authors
 98% 556M/565M [00:07<00:00, 34.5MB/s]
100% 565M/565M [00:07<00:00, 74.9MB/s]


In [ ]:
! unzip fashion-product-images-small.zip

Streaming output truncated to the last 5000 lines.
  inflating: myntradataset/images/58131.jpg  
  inflating: myntradataset/images/58132.jpg  
  inflating: myntradataset/images/58133.jpg  
  inflating: myntradataset/images/58135.jpg  
  inflating: myntradataset/images/58136.jpg  
  inflating: myntradataset/images/58137.jpg  
  inflating: myntradataset/images/58138.jpg  
  inflating: myntradataset/images/58139.jpg  
  inflating: myntradataset/images/5814.jpg  
  inflating: myntradataset/images/58140.jpg  
  inflating: myntradataset/images/58141.jpg  
  inflating: myntradataset/images/58143.jpg  
  inflating: myntradataset/images/58144.jpg  
  inflating: myntradataset/images/58145.jpg  
  inflating: myntradataset/images/58146.jpg  
  inflating: myntradataset/images/58147.jpg  
  inflating: myntradataset/images/58148.jpg  
  inflating: myntradataset/images/58149.jpg  
  inflating: myntradataset/images/5815.jpg  
  inflating: myntradataset/images/58150.jpg  
  inflating: myntradataset/imag

#Importing Libraries


In [ ]:
#librarynya blm lengkap ya masih pake yang sekiranya perlu dulu
#ntar tambahin aja
import pandas as pd
import numpy as np
import os
import seaborn as sns
import shutil
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import plotly.express as px
import zipfile
import tensorflow as tf
import urllib.request
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from tensorflow import keras
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from os import listdir
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [ ]:
base_dir = '/content/myntradataset'
image_files_dir = os.path.join(base_dir, 'images')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# Menghapus direktori train, validation, dan test jika sudah ada
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
if os.path.exists(validation_dir):
    shutil.rmtree(validation_dir)
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)

# Membuat ulang direktori train, validation, dan test
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

BATASSSSS

In [ ]:
# Path dataset
base_dir = '/content/myntradataset'
image_files_dir = os.path.join(base_dir, 'images')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Load dataset
styles = pd.read_csv("/content/myntradataset/styles.csv", on_bad_lines='skip')
styles['filename'] = styles['id'].astype(str) + '.jpg'
styles = styles.dropna()
styles = styles[styles['masterCategory'] == 'Apparel']
styles = styles.dropna()

# Memastikan file gambar ada
styles['present'] = styles['filename'].apply(lambda x: os.path.exists(os.path.join(image_files_dir, x)))
styles = styles[styles['present']].reset_index(drop=True)

# Kategori yang diinginkan
desired_article_types = ['Tshirts', 'Jeans', 'Dresses', 'Jackets', 'Shirts', 'Tops', 'Sweaters', 'Skirts']
filtered_styles = styles[styles['articleType'].isin(desired_article_types)]

# Memastikan setiap tipe artikel memiliki minimal 2 gambar
article_type_counts = filtered_styles['articleType'].value_counts()
valid_article_types = article_type_counts[article_type_counts >= 2].index
filtered_styles = filtered_styles[filtered_styles['articleType'].isin(valid_article_types)]

# Membagi dataset menjadi set pelatihan, validasi, dan pengujian
train_data, temp_data = train_test_split(filtered_styles, test_size=0.3, stratify=filtered_styles['articleType'], random_state=42)

temp_data_article_counts = temp_data['articleType'].value_counts()
valid_temp_article_types = temp_data_article_counts[temp_data_article_counts >= 2].index
temp_data = temp_data[temp_data['articleType'].isin(valid_temp_article_types)]

val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['articleType'], random_state=42)

# Membuat subdirektori berdasarkan articleType
for article_type in desired_article_types:
    os.makedirs(os.path.join(train_dir, article_type), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, article_type), exist_ok=True)
    os.makedirs(os.path.join(test_dir, article_type), exist_ok=True)

# Fungsi untuk memindahkan file
def move_files(data, target_dir):
    for _, row in data.iterrows():
        src_path = os.path.join(image_files_dir, row['filename'])
        dst_path = os.path.join(target_dir, row['articleType'], row['filename'])
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)

# Memindahkan gambar ke folder train, validation, dan test
move_files(train_data, train_dir)
move_files(val_data, validation_dir)
move_files(test_data, test_dir)

# Mengurangi ukurannya karena terlalu besar
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_set = val_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Model CNN
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[IMAGE_SIZE[0], IMAGE_SIZE[1], 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=512, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=len(desired_article_types), activation='softmax'))

cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Training model
history = cnn.fit(
    training_set,
    steps_per_epoch=training_set.samples // BATCH_SIZE,
    validation_data=validation_set,
    validation_steps=validation_set.samples // BATCH_SIZE,
    epochs=50
)

# Evaluasi model
Y_pred = cnn.predict(validation_set)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_set.classes, y_pred))


Found 9636 images belonging to 8 classes.
Found 2065 images belonging to 8 classes.
Found 2066 images belonging to 8 classes.
Epoch 1/50
301/301 [==============================] - 85s 257ms/step - loss: 1.1263 - accuracy: 0.6090 - val_loss: 0.6086 - val_accuracy: 0.7686
Epoch 2/50
301/301 [==============================] - 79s 261ms/step - loss: 0.5937 - accuracy: 0.7813 - val_loss: 0.4813 - val_accuracy: 0.8359
Epoch 3/50
301/301 [==============================] - 74s 245ms/step - loss: 0.4836 - accuracy: 0.8210 - val_loss: 0.4206 - val_accuracy: 0.8530
Epoch 4/50
301/301 [==============================] - 74s 246ms/step - loss: 0.4172 - accuracy: 0.8478 - val_loss: 0.4323 - val_accuracy: 0.8530
Epoch 5/50
301/301 [==============================] - 73s 243ms/step - loss: 0.3649 - accuracy: 0.8635 - val_loss: 0.4274 - val_accuracy: 0.8555
Epoch 6/50
301/301 [==============================] - 75s 248ms/step - loss: 0.3415 - accuracy: 0.8738 - val_loss: 0.4122 - val_accuracy: 0.8682
Epoc

In [ ]:
cnn.save('cnn_model.h5')

loaded_model = tf.keras.models.load_model('cnn_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
##UJI GAMBAR

import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Path gambar uji
img_path = '/content/myntradataset/WhatsApp Image 2024-06-18 at 00.12.55.jpeg'

img = image.load_img(img_path, target_size=(150, 150))

plt.imshow(img)
plt.axis('off')
plt.show()

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

predictions = cnn.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
class_labels = list(training_set.class_indices.keys())
predicted_label = class_labels[predicted_class[0]]

print(f'Prediksi: {predicted_label}')